## INTRODUCTION

Inspired by the wave analyses and manipulations that homework 9 required, as well as by our own experiences as musicians, we chose to create what we are calling an autosampler. In a musical context, sampling is the act of selecting and isolating small parts (known as samples) of a recording, and using those samples as part of a new piece of music. Sampling has always been common in the world of hip hop, where drum beat samples are looped and used as a backing track. More recently, some producers of electronic dance music have risen to prominence whose music consists almost entirely of short samples, mixed together and played as their own instrument. In either case, the producer needed to manually create his own samples. Using modern-day software, this is done by simply selecting a starting and an ending point on a recording's sound wave, and taking the part of the wave that lies between those two points. While this process sounds simple enough, it becomes quite tedious when you have large numbers of recordings to analyze, listen to for potential samples, and cut samples from. Our goal in this project is to automatically cut samples from user-provided wave files, and arrange those samples to produce output in the tune of a user-provided melody. For the examples presented in this paper, we have used the melody of Beethoven's well known piece Fur Elise.

## CODE DESCRIPTION

The meat of this project is the functionality of chopping up an input wave into samples, and then properly categorizing and storing the resulting pieces. In our initial, naive approach, we decided to chop the waves such that one sample would have the length of an eighth note, given a predefined tempo in beats per minute. We arbitrarily chose a tempo of 148 beats per minute. This translated to approximately 2.467 beats per second, or 0.2027 seconds per note since the eighth note is traditionally half a beat. Using a standard sampling rate of 44.1 kHz, this meant that each note would be 8939 data points long.

With this information, the next step is to iterate over each input wave provided by the user. Using the functions from waveIO.py, each file is read in and unpacked.The wave is then split, as mentioned above, into pieces 8939 data points long. This list of points serves as the samples that will be analyzed, sorted, and built into a final output wave.

This list of samples is then iterated over for sorting and storage. Samples shorter than 8939 samples long are discarded, which prevents shortened samples such as those that can occur at the end of a wave from being sorted with the samples of uniform length. The Fourier Transform of the sample is taken using Numpy's fft module, and then divided by the length of the sample in data points to normalize its amplitudes. Numpy'sfftfreq function is used to create the frequency bins for the sample, and the sample's strongest frequency is found.

This is where the amplitude normalization becomes important. The amplitude of the strongest frequency is compared to a predefined threshold value. If the amplitude is lower than this threshold, then no frequencies are extracted from the sample. If the amplitude is greater than the threshold, the sample's data is added to a large list of samples, and its index within that list is noted. The sample proceeds with sorting. In that case, the frequency is compared with a dictionary containing baseline frequencies for the twelve semitones in a musical octave. Since the A4 (this notation stands for the note A in the fourth octave, with A being the tenth not in the octave) note with an ideal frequency of 440.0 Hz is typically used as a standard, I used the frequencies of all the fourth octave notes to populate this dictionary.

If the sample's frequency is lower than the lowest frequency in the dictionary (with about 1% tolerance), then it must be part of a lower octave. Likewise, if it is higher than the highest frequency in the dictionary (with the same tolerance), it must be part of a higher octave. In the former case, I divide a multiplier value (initially set to 1) by 2 and compare the sample frequency to the highest and lowest frequencies in the dictionary, multiplied by the multiplier value. If the frequency is still too low, I again halve the multiplier, continuing this process until the sample frequency falls between the highest and lowest dictionary frequencies times the multiplier. Alternatively, in the case that the sample frequency was too high, I perform the same process, but double the multiplier instead of halving it on each unsuccessful comparison.

Having ascertained the octave of the sample frequency, and with specific values of the multiplier corresponding to specific octaves, the sample can be properly stored. The dictionary of note frequencies is iterated over, with each note's frequency value multiplied by the multiplier from earlier. If the sample frequency is within 1% of this multiplied dictionary value, its index in the larger sample list is stored in a list within a dictionary of the following structure:

In [1]:
notes_db = {
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[],[],[]],
"G#":[[],[],[],[],[],[]]
}


Where each note has a list of sub-lists, and each sub-list holds the index values of samples that contain the corresponding note in a specific octave.

If the code were to stop at this point, it could deal perfectly fine with simple input waves of single notes. Chords, however, would pose a problem, as only one frequency is pulled from each sample. To solve this problem, after the sample is sorted into one of the bins above, the strongest frequency is deleted from the Fourier Transform data, as is the frequency bin that corresponded to that frequency. The octave multiplier is reset to 1, and the next strongest frequency is pulled from the Fourier data. Its amplitude is once again compared to the threshold, and the whole process repeats itself, selecting, sorting, and deleting the strongest frequencies from the sample until the strongest frequency is lower than the threshold. Once this happens, the sample is completely sorted, and the sample's index is stored in bins for each note that was contained within the sample. It is important to note that on these subsequent loops, a frequency will only be added to a bin if the same sample's index does not already appear in that bin. This prevents a sample from being sorted twice into the same note's bin.

For example, after completely sorting a sample of a C Major chord in the fourth octave, where all three notes were above threshold and there was no noise above the threshold, the notes_db structure would look like this:

In [ ]:
{
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[sample_index],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[sample_index],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[sample_index],[],[]],
"G#":[[],[],[],[],[],[]]
}

Once all the samples from a given input wave are sorted in this manner, it is time to assemble them into the music the user provided. A sample of the notation used to specify the output music is shown below.

This example consists of the first few measures of Fur Elise. The notes to be played at a given time appear on each line. "#" symbols are used to denote boundaries between measures for convenience of reading and writing, and are ignored by the program. "%" symbols denote rests, or periods where no notes are played. Each note or rest has the length of one sample. Chords can be denoted by having multiple notes on the same line, separated by commas.

First an empty array is initialized to hold the output wave. Each line of the input music file is isolated and split by comma characters. If the line consists of a "#" symbol, the line is ignored and the next line is processed. If the line consists of a "%" symbol, a sample-length set of zeros is generated and appended to the output wave. Otherwise, for each note symbol on the line, a random sample index is pulled from the note's corresponding bin. The sample whose index was chosen is appended onto the output wave.

Once all the lines have been processed, the output wave is packed and saved under a user-provided filename, using functions from waveIO.


When I transitioned from my naive approach to a better representation of the STFT, I defined two constants, `WINDOW_LENGTH` and `OVERLAP_PERCENT`. These replaced the tempo, bpm, and dt calculations my naive version had done. `WINDOW_LENGTH` was the length, in data points of the STFT sampling window. `OVERLAP_PERCENT` is how much overlap there will be between consecutive windows. A value ov 0.5 would mean the second window would start at the midpoint of the first, and so on. I also created two functions, `window_rectangular` and `window_hanning`, which apply their given window functions to a `WINDOW_LENGTH` wave segment.

Rather than cutting the wave up into equal size segments, the wave is iterated over, with a step size of `WINDOW_LENGTH*(1-OVERLAP_PERCENT)`. In each step, a `WINDOW_LENGTH` sized portion is selected, and the `WINDOW_FUNC` is called upon that segment. Once this is done, the segment is added to an array of samples from the input wave, just as it had been in the naive version

## Physical Description

In the realm of soundwave analysis, there are a number of "perspectives" from which a wave can be viewed. The first of these, and likely the most familiar to many people, is the time domain perspective. The time domain perspective for a given soundwave shows the wave's amplitude over time. In music production, this perspective is often used to visually identify buildups, musical climaxes, and changes from one section of a song to another. It is this perspective which is typically used when a visual representation of a sound wave is needed.

<img src="Results/figures/vocal_scales_time_domain.png">[Fig 1]

The figure above is an example of the time perspective. From this perspective, it is easily seen that there are four distinct sections, each separated by short segments of much lower volume. However, little information about the contents of those louder sections can be gathered from this view.

One aspect of the wave which this view gives no information on, but which is very important for musical application is frequency. The frequency of a wave determines the pitch of the sound a wave makes. The higher the wave's frequency, the higher pitched its note will sound. Musical notes are organized into sets called octaves. An octave consists of twelve different frequencies, called semitones, that represent musical notes. The lowest note in an octave is C, the highest is B. The standard notation "C4" stands for the C note in the fourth octave, which has a frequency of 261.63 Hz. To find the frequency of a note *n* semitones higher than frequency *f*, the following formula is used:

<b><i>f_new = f \* 2<sup>n/12</sup></i></b>

From this formula, the conclusion can be drawn that a note exactly one octave, or twelve semitones, above frequency *f* has the frequency *2f*:

<b><i>f_octave = f \* 2<sup>12/12</sup> = f \* 2 = 2f</i></b>

With this knowledge, the next logical perspective to look at is the frequency perspective. This perspective is used to determine what frequencies are present in a given wave. To obtain this perspective, the Fourier Transform is performed on the wave. With the resulting amplitudes normalized by dividing them by the number of samples in the wave, the result looks like this:

<img src="Results/figures/sample_wave_ps.png">[Fig 2]

This example shows the frequency perspective of a simple, computer generated wave. Clearly there are three main frequencies present. They are at approximately 440 Hz, 550 Hz, and 660 Hz. The wave used to generate this spectrum consisted of short segments where each of those frequencies (corresponding to A4, C#5, and E5, respectively, the three notes in an A Major chord) is played individually, then a final segment where the three were played together. A second sample wave was generated in which the same notes were played in reverse order, before again finishing with all three played together. Observe the frequency domain of this sample:

<img src="Results/figures/sample_wave_ps_reverse.png">[Fig 3]

The two plots look identical! This illustrates an important aspect of the frequency domain: just as the time domain conveyed no usable information about the frequencies contained within its wave, the frequency domain conveys no information about the times at which its frequencies occur.

Ideally, there would be a perspective that can show both frequency and time simultaneously. This way, we could isolate exactly what notes are happening and when. Such a perspective would then allow for hand picking sections of the wave to take samples from, if manual sampling were the goal. As it happens, there is a perspective that does exactly this: the spectrogram. A spectrogram is a plot of frequency strength over time. The horizontal axis typically represents time, while the vertical represents various frequencies. to denote amplitude of a frequency at a given time, the corresponding point on the plot is colored. In the following examples, dark red indicates high amplitude at the frequency and time, and yellow indicates low amplitude. The following are spectrograms provided by matplotlib for the two sample waves used for the power spectra above:

<img src="Results/figures/sample_wave_specgram.png">[Fig 4]

<img src="Results/figures/sample_wave_specgram_reverse.png">[Fig 5]

In these spectrograms, the red bands center around the frequencies being played at a given time in the wave. It is clear, looking at them, that the two samples contain the same notes, being played in a different order, and it is clear exactly what that order is for each sample.

The spectrograms above were generated easily from wave data using matplotlib's `specgram` function, however the method by which the function generates them is the key to this project. `Specgram` makes use of what is known as the **Short Time Fourier Transform**, or STFT. The STFT is defined as follows:

<img src="STFT equation.png">[Fig 6, Bebis]

STFT essentially takes the Fourier Transform of very small parts of the wave at a time. This is done by multiplying the wave by a windowing function, which is nonzero only in a small time range centered around time `t'`, and then taking the Fourier Transform of the result. These small transforms are taken over the entire length of the wave by incrementally increasing `t'`, and a matrix can be created showing the intensity of all frequencies in all the small parts of the wave. These matrices can be plotted as the spectrograms seen above.

Looking again at the spectrograms, there is one key difference other than simply the order in which the notes are presented. The thicknesses of the frequency bars are different in the two figures. This is a factor of how large the STFT window is. A longer STFT window provides greater frequency precision, showing more clearly exactly what frequencies are represented. There is a tradeoff here, though. Imagine an STFT whose window width was equal to the length of the entire wave. This would be the same as a normal Fourier Transform! That is to say we would lose all indication of the frequencies' timing! The frequency resolution given by STFT with a given window length L is defined by:

$\Delta F = F_{s} / L Hz$

Conversely, imagine an STFT with an infinitely short windowing function such as the Dirac Delta Function:

<img src="STFT infinite short.png">[Fig 7, Bebis]

As you can see, because this windowing function gives a result of zero for all points other than `t'`, the result of the STFT would just be a time spectrum of the wave, losing all frequency representation (Bebis).


For the purposes of this project, I am not trying to build a spectrogram, however, but rather take apart the wave into small pieces (samples), which can then be put together in any order to create a new piece of choesive music. The principle, however, remains the same.


## Results

I began by building a naive version of the STFT (`sampler-naive.py`). This chopped up the input waves into equally sized chunks (in this case 8939 data points long), and then computed the Fourier Transform on each chunk to give myself a set of equally sized samples with which to build my output wave. This is equivalent to performing an STFT with window size 8939, and $\Delta_{t'} = 4469$. What this means is that there is no overlap between the STFT windows; the next window begins where the previous window ended. I then created a wave file which consisted of a chromatic scale from C4 to G#6. This wave was called `bigsample.wav`, Every note in that range was represented, which made it ideal for sampling and putting together into a new piece.

I ran `bigsample.wav` through my naive function, which chopped it up successfully, sorted the samples into note bins (a threshold of 1400 was used) and rearranged the samples into `furelise-naive.wav`. I also plotted how many samples per note between C4 and G#6 the function was able to get from the wave. That plot is below:

<img src="Results/figures/note_numbers_naive.png">[Fig 8]

There are at least two samples per note, with some notes having three. The reason some notes have an extra sample has to do with sample windows which lie on boundaries between two notes playing. These boundaries would count as a sample for both of the two notes contained within the window. Listening to `furelise-naive.wav` confirms this. Every so often a sample is heard that sounds like two notes, each a fraction of the normal note length. These are those boundary-case samples. Comparing the spectrogram of `furelise-naive.wav` to that of an analytically generated `analytic_furelise.wav` files, we can see these boundary cases causing some discrepancies, namely around the 2.25 second mark: 

<img src = "Results/figures/analytic_furelise_specgram.png">[Fig 9]

<img src = "Results/figures/naive_furelise_specgram.png">[Fig 10]

With a better understanding of STFT, I restructured my naive code to make manipulating the parameters of the STFT more manageable. The result was `sampler-stft.py`. To ensure that it was working as desired, I set it up with the same initial conditions as my naive approach had used: window size set to 8939, window step set to the window size, and using the same `bigsample.wav` as the input wave. The plot of samples per note is identical to that generated by my naive model:

<img src="Results/figures/note_numbers_stft_noverlap.png">[Fig 11]

To reduce the boundary cases that make it into the output wave, it makes sense to shorten the window length. While the number of boundary cases is likely to stay the same or even increase slightly (There are only so many note boundaries in the wave to begin with), the number ofsamples pulled from the file will increase more so that the boundary cases are less likely to be selected. Using a  window length of 4500, the samples per note plot changes in a couple notable ways:

<img src="Results/figures/note_numbers_smallwindow.png">[Fig 12]

As expected the number of samples has roughly doubled. Most notes now have 4 samples instead of two, with some notes having five. More importantly, though, C4, C#4, and E4 have no samples! There are two factors at play here: frequency resolution, and tolerance in my note sorting algorithm. By reducing my window size, I have also reduced my frequency resolution from approximately 4.9 Hz to 9.8 Hz. With this lower frequency resolution, I need to raise my tolerance to be less restrictive when sorting notes. Raising this tolerance from 1% to 2% gives the following plot as a result:

<img src="Results/figures/note_numbers_smallwindow_higher_tolerance.png">[Fig 13]

The notes that were missing from the prior plot are now present again. Arranging these samples into the Fur Elise email resulted in `furelise-stft-doubletol.wav`. Listening to this sample provides litle information on whether the boundary conditions are rarer, however, because the shorter window led to shorter samples. With samples this short, it is hard to differentiate full-note samples from samples that sound shorter due to being comprised of two notes separated by a note boundary. What makes it difficult is the fact that there is an audible "click" sound when one sample stops and another begins. This is because there is an amplitude discontinuity at the point where the two samples meet.

To reduce this issue, we use a different windowing function. Rather than a simple rectangular function, we use a Hanning function, which is a modified cosine function. The Hanning function looks like this:

<img src="hanning.png">[Fig 14: from www.diracdelta.co.uk]

Using the Hanning function as the windowing function ensures that the beginning and end of all samples will be zero, and will eliminate the harsh amplitude jumps that can happen when two samples are out of phase with one another. Since this reduces the amplitude of all frequencies except those in the very middle of the STFT window, this has the side effect of reducing the magnitude of each frequency returned by the Fourier Transform. For this reason, threshold values that may have worked for the rectangular window function can be too high, resulting in no samples sorted at all. Using this hanning function on `bigsample.wav` with window length of 4500 samples, 2% tolerance, and varying threshold values revealed that 1067 was the maximum threshold that could be used which would return samples for all notes in the sample, while 895 was the minumum threshold that would return as many samples as were returned by the rectangular window function with threshold 1400 and the other initial conditions the same.

The following is a spectrogram of a Fur Elise melody generated from `bigsample.wav` using the Hanning window function with threshold 895 and window size 8939 (`furelise_hanning_8939_895.wav`):

<img src="Results/figures/furelise_hanning_8939_specgram.png">[Fig 15]

Comparing to Figures 9 and 10, the most noticeable difference is the lack of vertical red bars between each sample. Those vertical red bars are the "clicks" that were heard at the boundary of the samples. The Hanning window function has served its purpose! Notice that there are still a couple of those vertical bars. Listening to `furelise_hanning_8939_895.wav` confirms that there are still a couple of audible clicks. These, however, are simply artifacts of the source `bigsample.wav` wave. This source did not fade notes into one another, but rather abruptly cut them off, creating the same kinds of amplitude discontinuities that were previously seen in the rectangular windowed STFT spectrograms. To confirm this, I created a new version of `bigsample.wav` called `bigsample_hanning.wav`. In this file, I used the same hanning function on each note to smooth the boundaries between them. Running the sampler (with the most recently mentioned conditions) on `bigsample_hanning.wav` gave the following spectrogram:

<img src="Results/figures/furelise_hanning_samplehanning_specgram.png">[Fig 16]

As expected, the "clicks" are gone! This is confirmed by listening to `furelise_hanning_8939_895_samplehanning.wav`.


These results are what I was hoping for, but they came from very simple, computer-generated audio. This audio has no noise or harmonics the way that real-world audio would. The next step was to test my sampler with recordings of real instruments and voices. I found a recording of a three octave chromatic scale played on the piano to start with (`piano_three_octaves.wav`). The spectrogram of this input wave looks like this:

<img src="Results/figures/piano_3_octaves_specgram.png">[Fig 17]

From this spectrogram, you can clearly see the gradually rising pitch of the notes being played, but you can also see weaker frequencies appearing above each root note. Listening to the sample suggests only one note is played at a time. Those higher frequencies are **harmonics**. The piano, along with most other instruments who produce their sound via a vibrating cord produces frequencies called harmonics with every note played. These harmonc frequencies occur at multiples of the fundamental frequency (also called the first harmonic). For example, the second harmonic of a 440 Hz fundamental frequency would be 880 Hz. These harmonics have interesting effects on the output wave the sampler generates. Starting with the naive sampler with note length 8939 data points, amplitude threshold 500, and frequency tolerance at 1%, the following spectrogram is generatedby the resulting Fur Elise melody, with samples taken from `piano_three_octaves.wav`:

<img src="Results/figures/furelise_naive_piano.png">[Fig 18]

Figure 18 looks almost nothing like the spectrograms of the same melody shown in Figures 15 and 16! Listening to the `furelise_naive_piano.wav` wave file also sounds very much unlike the melodies made of computer generated sounds. The notes, for the most part, sound correct, however they are played in the wrong octave at times, leading to a much different sound. The reason for this is because of those harmonics! Take a look at the first note in Figure 16 and compare it to the first note in Figure 18. This first note is supposed to be an E5, which has a frequency of approximately 659 Hz. In Figure 16, this area is clearly shown as being the strongest frequency of the first note. In Figure 18, however, that frequency range, while certainly strong, is not nearly the strongest frequency. It is merely a harmonic of the lower octave E4 note. The same is true for many of the other notes that sound "off" in the melody. They are the same notes, at different octaves.

The same thing happens, for the same reasons, when we apply the Hanning-Windowed STFT to `piano_three_octaves.wav` (with the threshold lowered to 310). As in the computer-generated realm, using the Hanning function rids the final output (`furelise_piano_hanning.wav`) of the clicking sounds between samples (See spectrogram in Figure 19). Listening to the output wave, however, it is hard to even discern that the source instrument is a piano. The short, uniform length segments with equal attack and decay are incredibly unlike the instrument's normal sounds, which typically emplys high sustain and a significantly longer decay than attack.

<img src="Results/figures/furelise_piano_hanning_specgram.png">[Fig 19]

As one final attempt to get a "natural" sounding piano wave, I again tweaked my appraoch to STFT. I set the `OVERLAP_PERCENT` to 0.5, meaning samples pulled from the input wave would overap with half of their preceding sample. Then I also made slight changes to the way the output wave is built. Since the Hanning window function is a modified cosine, I also had my output samples overlap one another. In other words, the first half of the second sample was added to the second half of the first sample, and so on. By balancing the attack of the second sample with the decay of the first sample, I was able to eliminate the unnatural silences between notes in `furelise_piano_hanning.wav`. This, more natural output wave was called `furelise_piano_hanning_overlap.wav`. Its spectrogram can be seen in Figure 20.

<img src="Results/figures/furelise_piano_hanning_overlap_specgram.png">[Fig 20

Finally, I ran through the same set of experiments using a recording of female vocals. The recording I used as input here (`vocalscales.wav`) provides a diverse set of frequencies in the same range as those I have been working with, however it misses a couple of the lower notes in the 4th octave. Rather than fill that gap in with computer generated samples, I simply changed the notes in my music file (`furelise.txt`) so that the C4 notes, which are not present in the input wave, are replaced with C5 notes. Figures 19 and 20 show the spectrograms generated by the naive and the hanning STFT samplers, respectively. Both used window sizes of 8939 data points. The naive sampler used a threshold of 1000, the Hanning sampler used a values of 500. Both used tolerances of 1%.


<img src="Results/figures/furelise_naive_vocals_specgram.png">[Fig 21]

<img src="Results/figures/furelise_hanning_vocal_specgram.png">[Fig 22]

Listening to the output wave (`furelise_vocal_hanning.wav`), parts of the melody sound correct, others sound very incorrect. Looking at the spectrogram makes it clear why this is: some of the notes are changing frequency drastically during the note's length! The first note, for example, increases in frequency drastically! This results in a sliding sound, which is clearly not what is needed for a musical sampling application.

## Discussion

**Computer Generated input samples:**
The results of my experiments using computer generated samples were more of a baseline to familiarize myself with the process than anything else. The results that I got were consistent with what I expected. With total control over the structure of the input waves (what frequencies were represented, how many were represented at once, etc), as well as the lack of noise and harmonics inherent to pure sinusoidal data, I was able to construct input waves to serve my purposes perfectly. `bigsample.wav` is a perfect example of this. I needed a long sample that contained every note in a relatively large frequency range, so I just build a chromatic scale sample.

**Piano input sample:**
I knew going into this project that transitioning from computer generated input waves to real-world instrumental recordings would be a pretty large hurdle. Dealing with the harmonics generated by the piano was surprisingly difficult. The lower notes had such strong harmonic frequencies that they would often get sorted into not only their note bins, but also the note bins of their harmonics. This is what led to the output waves having low notes where they should not have been. I attempted to account for this by setting the threshold relative to the strongest frequency in a sample. My idea was that, in the low notes, the harmonics would fall under some ratio and not be counted in the note's sorting. For example, I would start with a base threshold of 50, but once I found the max amplitude in a sample, I would set the threshold to the maximum of 50, and a fraction of that amplitude. When the threshold was half the max amplitude, the higher notes were simply no longer represented at all! This was very surprising to me.

While I wasn't able to find a reliable way to deal with the harmonics, I was pleasantly surprised at how natural sounding an output wave I was able to generate. For the most part, if `furelise_piano_hanning_overlap.wav` were played for me without my knowing how it was made, I might think it was a normal piano recording.  This sample shows the potential of this auto-sampler for real use.

**Vocal input sample:**
In retrospect, I really set myself up for failure when it comes to vocal input because the sample I chose was a singer with a very pronounced vibrato. Unlike the piano or the computer-generated input waves I used, the vibrato, even when singing a single note, has a periodic frequency shift to it. This accounts for some of the sloping samples seen in Figures 21 and 22. On a whim, I tried shortening the window length to 4500, then using the overlapping Hanning method on the vocal sample. The result was worse than I expected, sounding more like a pen of chickens than any sort of music. If you are curious, that sample is called `furelise_vocal_hanning_overlap_low_threshold_half_window.wav`.

## Future Directions

There are a number of things I would like to do with this project in the future, and I do plan on continuing to work on it in my spare time. First, I want to continue researching harmonics, so that I can more accurately deal with the frequency range of a piano. I am also interested in testing the sampler with input from other instruments. I wasn't able to find a diverse enough sample of a brass instrument, for example, to try. I don't believe such instruments would have the same problem with harmonics, but I wonder if they would present their own challenges.

Another area I would want to look into is frequency filtering. Some of my output waves particularly from the piano input, had noticeable noise artifacts. Using frequency filtering I could likely get rid of these. Taking this a step further, frequency filtering could likely help to isolate a fundamental frequency from its harmonics. Finally, in a case that I truly wish I had gotten to explore in this project, filtering out frequencies could isolate one frequency band from a sample, allowing me to take apart chords, or even recordings of multiple instruments being played together!

From a computer science perspective, I think that it would be incredibly interesting to apply machine learning to this project. I imagine this would work by allowing the user to point out samples from the sampler's output wave that are either particularly good or particularly bad. Over time, the sampler would use this feedback to filter out samples that are likely to be deemed "bad." One case where this would be particularly useful is in the vocal sample. The user could single out those samples which exhibited heavy frequency sliding, and mark them as bad samples, so the sampler would learn to throw away samples with that quality..

Finally, once the sampler can adequately deal with harmonic frequencies and filter specific frequency bands from samples, I would love to experiment with mapping the sampler to a MIDI keybord. Feed the sample a large number of samples, they could be very similar or very diverse, and each keypress on the MIDI keyboard would trigger a sample filtered so that only that note's frequency range would play. This is very much a long way off, but is definitely the long term goal for a project like this.

## Bibliography

Bebis. Short Time Fourier Transform (STFT). Retrieved from University of Nevada, Reno Computer Science and Engineering department, via Google: www.cse.unr.edu%2F~bebis%2FCS474%2FLectures%2FShortTimeFourierTransform.ppt

Heinzel, Rüdiger, Schilling. "Spectrum and Spectral density estimation by the Discrete Fourier Transform (DFT), including a comprehensive list of window functions and some new flat-top windows." February 15, 2002. Max-Planck-Institute für Gravitationsphysik. December 8, 2015 <https://holometer.fnal.gov/GH_FFT.pdf>

"Piano Key Frequencies." Wikipedia. December 8, 2015. <https://en.wikipedia.org/wiki/Piano_key_frequencies>

Free Engineering Lectures. "The Short Time Fourier Transform | Digital Signal Processing." 2014. Web. 8 Dec. 2015. <https://www.youtube.com/watch?v=g1_wcbGUcDY>

"Piano Keys C3-C6". Posted on www.freesounds.com by user Tesabob2001 on October 21, 2013. <https://www.freesound.org/people/Tesabob2001/sounds/203494/>.

"tmp_Vocal Scales1729437025.mp3". Posted on www.freesounds.com by user tweedledee3 on January 29, 2014. <https://www.freesound.org/people/tweedledee3/sounds/216504/>.

## Code

My naive approach to STFT:
`

In [ ]:
#UPDATED FREQUENCIES


import numpy
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import waveIO
import random

sr = 44100.

bpm = 148
dt=0.5

threshold = 500

# This dict holds the "ideal" values for each note in the Fourth octave.
# A wave will be said to correspond to a given note if its frequency is within 1%
# of the ideal values in this table, multiplied appropriately to give the frequency for the correct octave
note_freqs = {
"A":440.,
"A#":466.16,
"B":493.88,
"C":261.63,
"C#":277.18,
"D":293.66,
"D#":311.13,
"E":329.63,
"F":349.23,
"F#":369.99,
"G":391.99,
"G#":415.31
}

#This will hold all our chopped up notes
# Each note consists of a list containing 5 sublists
# Each of those 5 sublists corresponds to an octave 1-5
# Octave 1 is the first element of the list, Octave 6 the sixth
# For example, notes_db["C"][2] will be a list of all wave chunks whose dominant frequency
# corresponds to a C3 note, that is, a C in the 3rd octave
notes_db = {
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[],[],[]],
"G#":[[],[],[],[],[],[]]
}


all_notes=[]

# Use a given tempo to determine the correct length of time for each chunk of wave
def compute_dt(bpm):
	#assuming dt is for an eighth note
	bps = bpm/60.
	spb = 1./bps
	dt = spb/2.
	return dt

# Split a long wave into chunks of note_size size
# note size is given in terms of list elements, so it must be computed beforehand
def split_wave(wave, note_size):
	chunks=[]
	for i in range(0, len(wave), note_size):
		chunks.append(wave[i:i+note_size])
	return chunks

#places a wave chunk in the correct bin
def store_note(chunk):
	octave_multiplier = 1

	w = numpy.fft.rfft(chunk) / (len(chunk))

	freqs = numpy.fft.fftfreq(len(w))
	idx = numpy.argmax(numpy.abs(w))
	frequency = freqs[idx] * (sr/2)

	if numpy.abs(w[idx]) > threshold and 27.5 <= frequency <= 4187:
		# store the note in the list of all stored chunk, and get its index within that list
		all_notes.append(chunk)
		note_index = len(all_notes) - 1

	#If the loudest frequency is greater than the threshold, store this chunk's index in the appropriate note bin
	# repeat for the next loudest frequency and so on until the frequencies are no longer louder than the threshold
	while numpy.abs(w[idx]) > threshold and 27.5 <= frequency <= 4187:
		#determine what octave then note is in
		# Doubling a frequency increases the note's octave by 1
		# Thus, if the frequency is outside the default range, simply half or double each bin value to figure out what note it is
		while frequency < (note_freqs["C"] * octave_multiplier)*0.99:
			octave_multiplier = octave_multiplier/2.
		while frequency > (note_freqs["B"] * octave_multiplier)*1.01:
			octave_multiplier = octave_multiplier*2

		# iterate over each note in the scale, testing the chunk frequency
		# If the chunk frequency is within 1% of the note's frequency for a given octave,
		# place the chunk's index into that octave's bin within the corresponding note bin
		# By storing indices rather than whole chunks here, we can avoid having to store multiple copies of chunks which
		# represent chords
		for note,note_frequency in note_freqs.iteritems():
			octave_frequency = note_frequency * octave_multiplier
			if octave_frequency * 0.99 <= frequency <= octave_frequency*1.01:
				if octave_multiplier==0.125 and note_index not in notes_db[note][0]:
					notes_db[note][0].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==0.25 and note_index not in notes_db[note][1]:
					notes_db[note][1].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==0.5 and note_index not in notes_db[note][2]:
					notes_db[note][2].append(note_index)
					#return [w,freqs]
				elif octave_multiplier==1 and note_index not in notes_db[note][3]:
					notes_db[note][3].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==2 and note_index not in notes_db[note][4]:
					notes_db[note][4].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==4 and note_index not in notes_db[note][5]:
					notes_db[note][5].append(note_index)
					#return [w, freqs]

		#delete this frequency and its data from w and freqs, then compute a new idx and frequency
		w = numpy.delete(w,idx)
		octave_multiplier = 1
		freqs = numpy.delete(freqs,idx)
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)



	
def print_notes():
	for i in range(6):
		note="C"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="C#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="D"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="D#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="E"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="F"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="F#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="G"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="G#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="A"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="A#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="B"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		


# Tests the sample wave I created in a different function. The first quarter of the file is just a ~440hz sin
# The second quarter is a roughly 554hz sin wave
# The third quarter is a roughly 659hz sin wave
def test_sample_wave(chunks):
	for i in range(len(chunks)/4):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["A"], rtol=0.01)
	for i in range(len(chunks)/4,len(chunks)/2):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["C#"], rtol=0.01)
	for i in range(len(chunks)/2,3*len(chunks)/4):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["E"], rtol=0.01)
	print("Tests passed for sample wave!")

def build_song(musicfile, dt):
	note_time = numpy.arange(0,dt, 1/sr)
	song_wave=numpy.array([])
	with open(musicfile,'r') as music:
		for line in music:
			line = line.strip()
			if line == '#':
				continue
			if line == '%':
				empty_sample = numpy.zeros((dt * sr) - 1)
				song_wave = numpy.append(song_wave, empty_sample)
				continue
			line_notes = line.split(',')
			for note in line_notes:
				note_name = note[:-1]
				note_octave = note[-1]

				note_sample_bin = notes_db[note_name][int(note_octave) - 1]
				note_sample_index = random.choice(note_sample_bin)
				note_sample = all_notes[note_sample_index]

				song_wave = numpy.append(song_wave, note_sample)


	return song_wave

def plot_notes():
	objects = []
	quantities = []
	for i in range(3,6):
		objects.append('C{!s}'.format(i+1))
		objects.append('C#{!s}'.format(i+1))
		objects.append('D{!s}'.format(i+1))
		objects.append('D#{!s}'.format(i+1))
		objects.append('E{!s}'.format(i+1))
		objects.append('F{!s}'.format(i+1))
		objects.append('F#{!s}'.format(i+1))
		objects.append('G{!s}'.format(i+1))
		objects.append('G#{!s}'.format(i+1))
		objects.append('A{!s}'.format(i+1))
		objects.append('A#{!s}'.format(i+1))
		objects.append('B{!s}'.format(i+1))

		quantities.append(len(notes_db['C'][i]))
		quantities.append(len(notes_db['C#'][i]))
		quantities.append(len(notes_db['D'][i]))
		quantities.append(len(notes_db['D#'][i]))
		quantities.append(len(notes_db['E'][i]))
		quantities.append(len(notes_db['F'][i]))
		quantities.append(len(notes_db['F#'][i]))
		quantities.append(len(notes_db['G'][i]))
		quantities.append(len(notes_db['G#'][i]))
		quantities.append(len(notes_db['A'][i]))
		quantities.append(len(notes_db['A#'][i]))
		quantities.append(len(notes_db['B'][i]))
	print(objects)

	y_pos = numpy.arange(len(objects))
	print(y_pos)
	print(quantities)
	plt.bar(y_pos, quantities, align='center')
	plt.title("notes pulled from wave by quantity, naive")
	plt.xlabel("Note names and octaves")
	plt.ylabel("quantity")
	plt.xticks(y_pos, objects)
	plt.show()


if __name__ == '__main__':
	global dt
	# figure out the correct dt length based on the tempo of the output wave you want
	dt = compute_dt(bpm)
	# read and parse each sample wave
	for i in range(1, len(sys.argv) - 2):
		#read in the wave file and unpack its data
		wave_data = waveIO.read_wav_file(sys.argv[i])
		wave_data = waveIO.unpack(wave_data)
		wave_time = len(wave_data)/sr

		note_length = dt * sr

		wave_chunks = split_wave(wave_data, int(note_length))

		#test_sample_wave(wave_chunks)

		for i in range(len(wave_chunks)):
			chunk = wave_chunks[i]
			if len(chunk) != int(note_length):
				continue

			store_note(chunk)
	print_notes()
	plot_notes()
	
	# read and parse the music file
	musicfile = sys.argv[-2]
	song = build_song(musicfile, dt)
	waveIO.write_wav_file(sys.argv[-1], waveIO.pack(song))

	

My window-based STFT approach:

In [ ]:
#UPDATED FREQUENCIES


import numpy
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import waveIO

sr = 44100.

bpm = 148
dt=0.5

threshold = 310
#threshold = 1067

WINDOW_LENGTH = 8939
OVERLAP_PERCENT = 0 # 0 <= OVERLAP PERCENT < 1

# This dict holds the "ideal" values for each note in the Fourth octave.
# A wave will be said to correspond to a given note if its frequency is within 1%
# of the ideal values in this table, multiplied appropriately to give the frequency for the correct octave
note_freqs = {
"A":440.,
"A#":466.16,
"B":493.88,
"C":261.63,
"C#":277.18,
"D":293.66,
"D#":311.13,
"E":329.63,
"F":349.23,
"F#":369.99,
"G":391.99,
"G#":415.31
}

#This will hold all our chopped up notes
# Each note consists of a list containing 5 sublists
# Each of those 5 sublists corresponds to an octave 1-5
# Octave 1 is the first element of the list, Octave 6 the sixth
# For example, notes_db["C"][2] will be a list of all wave chunks whose dominant frequency
# corresponds to a C3 note, that is, a C in the 3rd octave
notes_db = {
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[],[],[]],
"G#":[[],[],[],[],[],[]]
}


all_notes=[]

# Use a given tempo to determine the correct length of time for each chunk of wave
def compute_dt(bpm):
	#assuming dt is for an eighth note
	bps = bpm/60.
	spb = 1./bps
	dt = spb/2.
	return dt

# Split a long wave into chunks of note_size size
# note size is given in terms of list elements, so it must be computed beforehand
def split_wave(wave, note_size):
	chunks=[]
	for i in range(0, len(wave), note_size):
		chunks.append(wave[i:i+note_size])
	return chunks

#places a wave chunk in the correct bin
def store_note(chunk):
	global threshold
	#when beginning to store a new sample, reset the threshold to its initial value
	octave_multiplier = 1

	w = numpy.fft.rfft(chunk) / (len(chunk))

	freqs = numpy.fft.fftfreq(len(w))
	idx = numpy.argmax(numpy.abs(w))
	frequency = freqs[idx] * (sr/2)


	if numpy.abs(w[idx]) > threshold and 27.5 <= frequency <= 4187:
		# store the note in the list of all stored chunk, and get its index within that list
		all_notes.append(chunk)
		note_index = len(all_notes) - 1
		#set the threshold to a value relative to the maximum amplitude in this sample
		#threshold = 0.5 * numpy.abs(w[idx])

	#If the loudest frequency is greater than the threshold, store this chunk's index in the appropriate note bin
	# repeat for the next loudest frequency and so on until the frequencies are no longer louder than the threshold
	while numpy.abs(w[idx]) > threshold and 27.5 <= frequency <= 4187:
		#determine what octave then note is in
		# Doubling a frequency increases the note's octave by 1
		# Thus, if the frequency is outside the default range, simply half or double each bin value to figure out what note it is
		while frequency < (note_freqs["C"] * octave_multiplier)*0.99:
			octave_multiplier = octave_multiplier/2.
		while frequency > (note_freqs["B"] * octave_multiplier)*1.01:
			octave_multiplier = octave_multiplier*2

		# iterate over each note in the scale, testing the chunk frequency
		# If the chunk frequency is within 1% of the note's frequency for a given octave,
		# place the chunk's index into that octave's bin within the corresponding note bin
		# By storing indices rather than whole chunks here, we can avoid having to store multiple copies of chunks which
		# represent chords
		for note,note_frequency in note_freqs.iteritems():
			octave_frequency = note_frequency * octave_multiplier
			if octave_frequency * 0.99 <= frequency <= octave_frequency*1.01:
				if octave_multiplier==0.125 and note_index not in notes_db[note][0]:
					notes_db[note][0].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==0.25 and note_index not in notes_db[note][1]:
					notes_db[note][1].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==0.5 and note_index not in notes_db[note][2]:
					notes_db[note][2].append(note_index)
					#return [w,freqs]
				elif octave_multiplier==1 and note_index not in notes_db[note][3]:
					notes_db[note][3].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==2 and note_index not in notes_db[note][4]:
					notes_db[note][4].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==4 and note_index not in notes_db[note][5]:
					notes_db[note][5].append(note_index)
					#return [w, freqs]

		#delete this frequency and its data from w and freqs, then compute a new idx and frequency
		w = numpy.delete(w,idx)
		octave_multiplier = 1
		freqs = numpy.delete(freqs,idx)
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)



	
def print_notes():
	for i in range(6):
		note="C"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="C#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="D"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="D#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="E"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="F"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="F#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="G"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="G#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="A"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="A#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="B"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		


# Tests the sample wave I created in a different function. The first quarter of the file is just a ~440hz sin
# The second quarter is a roughly 554hz sin wave
# The third quarter is a roughly 659hz sin wave
def test_sample_wave(chunks):
	for i in range(len(chunks)/4):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["A"], rtol=0.01)
	for i in range(len(chunks)/4,len(chunks)/2):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["C#"], rtol=0.01)
	for i in range(len(chunks)/2,3*len(chunks)/4):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["E"], rtol=0.01)
	print("Tests passed for sample wave!")

def build_song(musicfile, dt):
	note_time = numpy.arange(0,dt, 1/sr)
	song_wave=numpy.array([])
	with open(musicfile,'r') as music:
		for line in music:
			line = line.strip()
			if line == '#':
				continue
			if line == '%':
				empty_sample = numpy.zeros((dt * sr) - 1)
				song_wave = numpy.append(song_wave, empty_sample)
				continue
			line_notes = line.split(',')
			for note in line_notes:
				note_name = note[:-1]
				note_octave = note[-1]

				note_sample_bin = notes_db[note_name][int(note_octave) - 1]
				note_sample_index = numpy.random.choice(note_sample_bin)
				note_sample = all_notes[note_sample_index]

				song_wave = numpy.append(song_wave, note_sample)


	return song_wave


def apply_window_function(func, data, tau):
	result = numpy.array([])
	for t in data:
		result = numpy.append(result, data[t] * func(t - tau))
	return result

#Since we only apply the function to a window-size piece at a time, the rectangular window function simply returns the window sized piece intact
def window_rectangular(wave_window):
	return wave_window*1

def window_hanning(wave_window):
	#get the values of a hanning curve. The wave window will be multiplied by these values
	hanning_multipliers = numpy.hanning(len(wave_window))
	result=[]
	for i in range(len(wave_window)):
		result.append(wave_window[i]*hanning_multipliers[i])
	return result

def plot_notes():
	objects = []
	quantities = []
	for i in range(3,6):
		objects.append('C{!s}'.format(i+1))
		objects.append('C#{!s}'.format(i+1))
		objects.append('D{!s}'.format(i+1))
		objects.append('D#{!s}'.format(i+1))
		objects.append('E{!s}'.format(i+1))
		objects.append('F{!s}'.format(i+1))
		objects.append('F#{!s}'.format(i+1))
		objects.append('G{!s}'.format(i+1))
		objects.append('G#{!s}'.format(i+1))
		objects.append('A{!s}'.format(i+1))
		objects.append('A#{!s}'.format(i+1))
		objects.append('B{!s}'.format(i+1))

		quantities.append(len(notes_db['C'][i]))
		quantities.append(len(notes_db['C#'][i]))
		quantities.append(len(notes_db['D'][i]))
		quantities.append(len(notes_db['D#'][i]))
		quantities.append(len(notes_db['E'][i]))
		quantities.append(len(notes_db['F'][i]))
		quantities.append(len(notes_db['F#'][i]))
		quantities.append(len(notes_db['G'][i]))
		quantities.append(len(notes_db['G#'][i]))
		quantities.append(len(notes_db['A'][i]))
		quantities.append(len(notes_db['A#'][i]))
		quantities.append(len(notes_db['B'][i]))

	y_pos = numpy.arange(len(objects))

	plt.bar(y_pos, quantities, align='center', width=0.75)
	plt.title("notes pulled from wave by quantity, stft")
	plt.xlabel("Note names and octaves")
	plt.ylabel("quantity")
	plt.xticks(y_pos, objects, fontsize=10)
	plt.show()


if __name__ == '__main__':
	WINDOW_FUNC = window_hanning

	# read and parse each input wave
	for i in range(1, len(sys.argv) - 2):
		#read in the wave file and unpack its data
		wave_data = waveIO.read_wav_file(sys.argv[i])
		wave_data = waveIO.unpack(wave_data)
		wave_time = len(wave_data)/sr


		wave_chunks = []

		#Instead of multiplying the entire wave by the windowing function, we can simply examine window-sized pieces of the wave at a time
		#counting by WINDOW_LENGTH*(1-OVERLAP_PERCENT) allows adjustment of what percent of each window overlaps with the next
		#Example: OVERLAP_PERCENT = 0 means the next window will start where the previous window ended
		#Example: OVERLAP_PERCENT = 0.5 means the next window will start at the halfway mark of the previous window
		#This is necessary when using nonrectangular window functions, to prevent loss of data
		for i in range(0, len(wave_data), WINDOW_LENGTH*(1-OVERLAP_PERCENT)):
			window = []
			window.extend(wave_data[i:i+WINDOW_LENGTH])
			#multiply by the window function
			window = WINDOW_FUNC(window)
			wave_chunks.append(window)
			
		for i in range(len(wave_chunks)):
			chunk = wave_chunks[i]
			if len(chunk) != WINDOW_LENGTH:
				continue

			store_note(chunk)
	print_notes()
	plot_notes()
	
	# read and parse the music file
	musicfile = sys.argv[-2]
	song = build_song(musicfile, WINDOW_LENGTH / float(sr))
	waveIO.write_wav_file(sys.argv[-1], waveIO.pack(song))

	

My overlapping window-based STFT:

In [ ]:
#UPDATED FREQUENCIES


import numpy
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import waveIO

sr = 44100.

bpm = 148
dt=0.5

threshold = 25
#threshold = 1067

WINDOW_LENGTH = 4500
OVERLAP_PERCENT = 0.5 # 0 <= OVERLAP PERCENT < 1

# This dict holds the "ideal" values for each note in the Fourth octave.
# A wave will be said to correspond to a given note if its frequency is within 1%
# of the ideal values in this table, multiplied appropriately to give the frequency for the correct octave
note_freqs = {
"A":440.,
"A#":466.16,
"B":493.88,
"C":261.63,
"C#":277.18,
"D":293.66,
"D#":311.13,
"E":329.63,
"F":349.23,
"F#":369.99,
"G":391.99,
"G#":415.31
}

#This will hold all our chopped up notes
# Each note consists of a list containing 5 sublists
# Each of those 5 sublists corresponds to an octave 1-5
# Octave 1 is the first element of the list, Octave 6 the sixth
# For example, notes_db["C"][2] will be a list of all wave chunks whose dominant frequency
# corresponds to a C3 note, that is, a C in the 3rd octave
notes_db = {
"A": [[],[],[],[],[],[]],
"A#": [[],[],[],[],[],[]],
"B": [[],[],[],[],[],[]],
"C": [[],[],[],[],[],[]],
"C#": [[],[],[],[],[],[]],
"D": [[],[],[],[],[],[]],
"D#": [[],[],[],[],[],[]],
"E": [[],[],[],[],[],[]],
"F": [[],[],[],[],[],[]],
"F#": [[],[],[],[],[],[]],
"G": [[],[],[],[],[],[]],
"G#":[[],[],[],[],[],[]]
}


all_notes=[]

# Use a given tempo to determine the correct length of time for each chunk of wave
def compute_dt(bpm):
	#assuming dt is for an eighth note
	bps = bpm/60.
	spb = 1./bps
	dt = spb/2.
	return dt

# Split a long wave into chunks of note_size size
# note size is given in terms of list elements, so it must be computed beforehand
def split_wave(wave, note_size):
	chunks=[]
	for i in range(0, len(wave), note_size):
		chunks.append(wave[i:i+note_size])
	return chunks

#places a wave chunk in the correct bin
def store_note(chunk):
	global threshold
	#threshold=50
	#when beginning to store a new sample, reset the threshold to its initial value
	octave_multiplier = 1

	w = numpy.fft.rfft(chunk) / (len(chunk))

	freqs = numpy.fft.fftfreq(len(w))
	idx = numpy.argmax(numpy.abs(w))
	frequency = freqs[idx] * (sr/2)


	if numpy.abs(w[idx]) > threshold and 27.5 <= frequency <= 4187:
		# store the note in the list of all stored chunk, and get its index within that list
		all_notes.append(chunk)
		note_index = len(all_notes) - 1
		#set the threshold to a value relative to the maximum amplitude in this sample
		#threshold = max(0.2 * numpy.abs(w[idx]), 50)
	print("Max amplitude is {!s} at frequency {!s}".format(numpy.abs(w[idx]), frequency))

	#If the loudest frequency is greater than the threshold, store this chunk's index in the appropriate note bin
	# repeat for the next loudest frequency and so on until the frequencies are no longer louder than the threshold
	while numpy.abs(w[idx]) > threshold and 27.5 <= frequency <= 4187:
		#determine what octave then note is in
		# Doubling a frequency increases the note's octave by 1
		# Thus, if the frequency is outside the default range, simply half or double each bin value to figure out what note it is
		while frequency < (note_freqs["C"] * octave_multiplier)*0.99:
			octave_multiplier = octave_multiplier/2.
		while frequency > (note_freqs["B"] * octave_multiplier)*1.01:
			octave_multiplier = octave_multiplier*2

		# iterate over each note in the scale, testing the chunk frequency
		# If the chunk frequency is within 1% of the note's frequency for a given octave,
		# place the chunk's index into that octave's bin within the corresponding note bin
		# By storing indices rather than whole chunks here, we can avoid having to store multiple copies of chunks which
		# represent chords
		for note,note_frequency in note_freqs.iteritems():
			octave_frequency = note_frequency * octave_multiplier
			if octave_frequency * 0.99 <= frequency <= octave_frequency*1.01:
				if octave_multiplier==0.125 and note_index not in notes_db[note][0]:
					notes_db[note][0].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==0.25 and note_index not in notes_db[note][1]:
					notes_db[note][1].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==0.5 and note_index not in notes_db[note][2]:
					notes_db[note][2].append(note_index)
					#return [w,freqs]
				elif octave_multiplier==1 and note_index not in notes_db[note][3]:
					notes_db[note][3].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==2 and note_index not in notes_db[note][4]:
					notes_db[note][4].append(note_index)
					#return [w, freqs]
				elif octave_multiplier==4 and note_index not in notes_db[note][5]:
					notes_db[note][5].append(note_index)
					#return [w, freqs]

		#delete this frequency and its data from w and freqs, then compute a new idx and frequency
		w = numpy.delete(w,idx)
		octave_multiplier = 1
		freqs = numpy.delete(freqs,idx)
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)



	
def print_notes():
	for i in range(6):
		note="C"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="C#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="D"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="D#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="E"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="F"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="F#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="G"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="G#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="A"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="A#"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		note="B"
		print("{!s}{!s}: {!s}".format(note, i+1, len(notes_db[note][i])))
		


# Tests the sample wave I created in a different function. The first quarter of the file is just a ~440hz sin
# The second quarter is a roughly 554hz sin wave
# The third quarter is a roughly 659hz sin wave
def test_sample_wave(chunks):
	for i in range(len(chunks)/4):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["A"], rtol=0.01)
	for i in range(len(chunks)/4,len(chunks)/2):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["C#"], rtol=0.01)
	for i in range(len(chunks)/2,3*len(chunks)/4):
		chunk = chunks[i]

		w = numpy.fft.rfft(chunk)
		freqs = numpy.fft.fftfreq(len(w))
		idx = numpy.argmax(numpy.abs(w))
		frequency = freqs[idx] * (sr/2)

		numpy.testing.assert_allclose(frequency, note_freqs["E"], rtol=0.01)
	print("Tests passed for sample wave!")

def build_song(musicfile, dt):
	note_time = numpy.arange(0,dt, 1/sr)
	song_wave=numpy.array([])
	with open(musicfile,'r') as music:
		for line in music:
			line = line.strip()
			if line == '#':
				continue
			for k in range(4):
				#increase the length of song wave by half a window
				if len(song_wave) >= WINDOW_LENGTH:
					song_wave = numpy.append(song_wave, numpy.zeros(WINDOW_LENGTH / 2))
				if line == '%':
					#empty_sample = numpy.zeros((dt * sr) - 1)
					if len(song_wave) < WINDOW_LENGTH:
						song_wave = numpy.append(song_wave, empty_sample)
					#else:
					#	song_wave[-WINDOW_LENGTH :] = song_wave[-WINDOW_LENGTH : ] + empty_sample
					
					continue
				line_notes = line.split(',')
				for note in line_notes:
					note_name = note[:-1]
					note_octave = note[-1]

					note_sample_bin = notes_db[note_name][int(note_octave) - 1]
					note_sample_index = numpy.random.choice(note_sample_bin)
					note_sample = all_notes[note_sample_index]

					print(len(song_wave))
					#song_wave = numpy.append(song_wave, note_sample)
					if len(song_wave) < WINDOW_LENGTH:
						song_wave = numpy.append(song_wave, note_sample)
					else:
						song_wave[-WINDOW_LENGTH :] = song_wave[-WINDOW_LENGTH : ] + note_sample


	return song_wave


def apply_window_function(func, data, tau):
	result = numpy.array([])
	for t in data:
		result = numpy.append(result, data[t] * func(t - tau))
	return result

#Since we only apply the function to a window-size piece at a time, the rectangular window function simply returns the window sized piece intact
def window_rectangular(wave_window):
	return wave_window*1

def window_hanning(wave_window):
	#get the values of a hanning curve. The wave window will be multiplied by these values
	hanning_multipliers = numpy.hanning(len(wave_window))
	result=[]
	for i in range(len(wave_window)):
		result.append(wave_window[i]*hanning_multipliers[i])
	return result

def plot_notes():
	objects = []
	quantities = []
	for i in range(3,6):
		objects.append('C{!s}'.format(i+1))
		objects.append('C#{!s}'.format(i+1))
		objects.append('D{!s}'.format(i+1))
		objects.append('D#{!s}'.format(i+1))
		objects.append('E{!s}'.format(i+1))
		objects.append('F{!s}'.format(i+1))
		objects.append('F#{!s}'.format(i+1))
		objects.append('G{!s}'.format(i+1))
		objects.append('G#{!s}'.format(i+1))
		objects.append('A{!s}'.format(i+1))
		objects.append('A#{!s}'.format(i+1))
		objects.append('B{!s}'.format(i+1))

		quantities.append(len(notes_db['C'][i]))
		quantities.append(len(notes_db['C#'][i]))
		quantities.append(len(notes_db['D'][i]))
		quantities.append(len(notes_db['D#'][i]))
		quantities.append(len(notes_db['E'][i]))
		quantities.append(len(notes_db['F'][i]))
		quantities.append(len(notes_db['F#'][i]))
		quantities.append(len(notes_db['G'][i]))
		quantities.append(len(notes_db['G#'][i]))
		quantities.append(len(notes_db['A'][i]))
		quantities.append(len(notes_db['A#'][i]))
		quantities.append(len(notes_db['B'][i]))

	y_pos = numpy.arange(len(objects))

	plt.bar(y_pos, quantities, align='center', width=0.75)
	plt.title("notes pulled from wave by quantity, stft")
	plt.xlabel("Note names and octaves")
	plt.ylabel("quantity")
	plt.xticks(y_pos, objects, fontsize=10)
	plt.show()


if __name__ == '__main__':
	WINDOW_FUNC = window_hanning

	# read and parse each input wave
	for i in range(1, len(sys.argv) - 2):
		#read in the wave file and unpack its data
		wave_data = waveIO.read_wav_file(sys.argv[i])
		wave_data = waveIO.unpack(wave_data)
		wave_time = len(wave_data)/sr


		wave_chunks = []

		#Instead of multiplying the entire wave by the windowing function, we can simply examine window-sized pieces of the wave at a time
		#counting by WINDOW_LENGTH*(1-OVERLAP_PERCENT) allows adjustment of what percent of each window overlaps with the next
		#Example: OVERLAP_PERCENT = 0 means the next window will start where the previous window ended
		#Example: OVERLAP_PERCENT = 0.5 means the next window will start at the halfway mark of the previous window
		#This is necessary when using nonrectangular window functions, to prevent loss of data
		for i in range(0, len(wave_data), int(WINDOW_LENGTH*(1-OVERLAP_PERCENT))):
			window = []
			window.extend(wave_data[i:i+WINDOW_LENGTH])
			#multiply by the window function
			window = WINDOW_FUNC(window)
			wave_chunks.append(window)
			
		for i in range(len(wave_chunks)):
			chunk = wave_chunks[i]
			if len(chunk) != WINDOW_LENGTH:
				continue

			store_note(chunk)
	print_notes()
	plot_notes()
	
	# read and parse the music file
	musicfile = sys.argv[-2]
	song = build_song(musicfile, WINDOW_LENGTH / float(sr))
	waveIO.write_wav_file(sys.argv[-1], waveIO.pack(song))

	

The script I used to generate `bigsample.wav` and similar:

In [ ]:
import numpy
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import waveIO
#import pyaudio

def create_wave(freq,t):
    return 5000 * numpy.cos(freq * 2 * numpy.pi * t)

def window_hanning(wave_window):
	#get the values of a hanning curve. The wave window will be multiplied by these values
	hanning_multipliers = numpy.hanning(len(wave_window))
	result=[]
	for i in range(len(wave_window)):
		result.append(wave_window[i]*hanning_multipliers[i])
	return result

sr = 44100.
dt = 1 / sr
NFFT = 1024
fs = 1.0/dt
t = numpy.arange(0,2,dt)
t_sin=create_wave(440 * 2**(7/12.), t)
third_sin=create_wave(440 * 2**(4/12.), t)
fifth_sin=create_wave(440, t)

chord_sin = t_sin + third_sin + fifth_sin

t_sin = numpy.append(t_sin, third_sin)
t_sin = numpy.append(t_sin, fifth_sin)
t_sin = numpy.append(t_sin, chord_sin)

t_sin_fft = (numpy.abs(numpy.fft.fft(t_sin)) / len(t_sin))
freqs = numpy.fft.fftfreq(len(t_sin_fft), 1/44100.)
idx = numpy.argsort(freqs)
frequency = freqs[idx] * (sr/2)
plt.plot(freqs[idx], t_sin_fft[idx])
plt.xlim(0,1000)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Absolute value of FFT Magnitude, divided by N")
#plt.yscale("log")
plt.title("Power spectrum of sample wave")
plt.show()


data, freqs, bins, im = plt.specgram(t_sin, NFFT=NFFT, Fs=fs, noverlap=NFFT/2)
plt.title("Spectrogram, notes in reverse, window size = 1024, sr = 44100")
plt.ylabel("Frequency (Hz)")
plt.xlabel("Time (s)")
plt.ylim(0,800)
plt.show()
#print(data)
waveIO.write_wav_file("samplewave.wav", waveIO.pack(t_sin))


##############
# Now build a universal sample, with short sections for each note through a few octaves
t = numpy.arange(0, 0.5, dt)
big_sample = create_wave(440, t)
big_sample = numpy.append(big_sample, create_wave(261, t))
big_sample = numpy.append(big_sample, create_wave(277, t))
big_sample = numpy.append(big_sample, create_wave(293, t))
big_sample = numpy.append(big_sample, create_wave(311, t))
big_sample = numpy.append(big_sample, create_wave(329, t))
big_sample = numpy.append(big_sample, create_wave(349, t))
big_sample = numpy.append(big_sample, create_wave(369, t))
big_sample = numpy.append(big_sample, create_wave(391, t))
big_sample = numpy.append(big_sample, create_wave(415, t))
big_sample = numpy.append(big_sample, create_wave(466.16, t))
big_sample = numpy.append(big_sample, create_wave(493, t))
big_sample = numpy.append(big_sample, create_wave(523, t))
big_sample = numpy.append(big_sample, create_wave(554, t))
big_sample = numpy.append(big_sample, create_wave(587, t))
big_sample = numpy.append(big_sample, create_wave(622, t))
big_sample = numpy.append(big_sample, create_wave(659, t))
big_sample = numpy.append(big_sample, create_wave(698, t))
big_sample = numpy.append(big_sample, create_wave(739, t))
big_sample = numpy.append(big_sample, create_wave(783, t))
big_sample = numpy.append(big_sample, create_wave(830, t))
big_sample = numpy.append(big_sample, create_wave(880, t))
big_sample = numpy.append(big_sample, create_wave(932, t))
big_sample = numpy.append(big_sample, create_wave(987, t))
big_sample = numpy.append(big_sample, create_wave(1046, t))
big_sample = numpy.append(big_sample, create_wave(1108, t))
big_sample = numpy.append(big_sample, create_wave(1174, t))
big_sample = numpy.append(big_sample, create_wave(1244, t))
big_sample = numpy.append(big_sample, create_wave(1318, t))
big_sample = numpy.append(big_sample, create_wave(1396, t))
big_sample = numpy.append(big_sample, create_wave(1479, t))
big_sample = numpy.append(big_sample, create_wave(1567, t))
big_sample = numpy.append(big_sample, create_wave(1661, t))
waveIO.write_wav_file("bigsample.wav", waveIO.pack(big_sample))


##############
# Now build a universal sample, with short sections for each note through a few octaves
t = numpy.arange(0, 0.5, dt)
big_sample = create_wave(440, t)
big_sample = window_hanning(big_sample)
big_sample = numpy.append(big_sample, window_hanning(create_wave(261, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(277, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(293, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(311, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(329, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(349, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(369, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(391, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(415, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(466.16, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(493, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(523, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(554, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(587, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(622, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(659, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(698, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(739, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(783, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(830, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(880, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(932, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(987, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1046, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1108, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1174, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1244, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1318, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1396, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1479, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1567, t)))
big_sample = numpy.append(big_sample, window_hanning(create_wave(1661, t)))
waveIO.write_wav_file("bigsample_hanning.wav", waveIO.pack(big_sample))

The script I used to create `analytic_furelise.wav`:

In [ ]:
import numpy
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import waveIO

def create_wave(freq,t):
    return 5000 * numpy.cos(freq * 2 * numpy.pi * t)

def window_hanning(wave_window):
	#get the values of a hanning curve. The wave window will be multiplied by these values
	hanning_multipliers = numpy.hanning(len(wave_window))
	result=[]
	for i in range(len(wave_window)):
		result.append(wave_window[i]*hanning_multipliers[i])
	return result

def build_song(musicfile):
	song_wave=numpy.array([])
	with open(musicfile,'r') as music:
		t = numpy.arange(0,0.2027,1/44100.)
		for line in music:
			line = line.strip()
			if line == '#':
				continue
			if line == '%':
				empty_sample = numpy.zeros(8938)
				song_wave = numpy.append(song_wave, empty_sample)
				continue
			line_notes = line.split(',')
			for note in line_notes:
				note_name = note[:-1]
				note_octave = note[-1]
				if note_octave == '4':
					song_wave = numpy.append(song_wave, window_hanning(create_wave(freqs4[note_name],t)))
				elif note_octave == '5':
					song_wave = numpy.append(song_wave, window_hanning(create_wave(freqs5[note_name],t)))
				elif note_octave == '6':
					song_wave = numpy.append(song_wave, window_hanning(create_wave(freqs6[note_name],t)))


	return song_wave

freqs4 = {
"A":440.,
"A#":466.16,
"B":493.88,
"C":261.63,
"C#":277.18,
"D":293.66,
"D#":311.13,
"E":329.63,
"F":349.23,
"F#":369.99,
"G":391.99,
"G#":415.31
}

freqs5 = {}
freqs6 = {}

note_length = 8938

if __name__ == '__main__':
	for note in freqs4:
		freqs5[note] = 2*freqs4[note]
		freqs6[note] = 4*freqs4[note]
	musicfile = sys.argv[1]
	musicwave = build_song(musicfile)
	data, freqs, bins, im = plt.specgram(musicwave, NFFT=2048, Fs=44100, noverlap=900)
	plt.ylim(0,800)
	plt.title("Spectrogram of analytic_furelise_hanning.wav")
	plt.show()
	waveIO.write_wav_file("analytic_furelise.wav", waveIO.pack(musicwave))